# Validation

In [5]:
import time
import warnings
from datetime import datetime
from functions import missing_value_handling, remove_multivariate_outlier, remove_univariate_outliers, uni_multi_variate_outlier, normalize, standardize, noise_filtering
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn import metrics

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.base import BaseEstimator
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor

from matplotlib import pyplot as plt
from sklearn.model_selection import GridSearchCV
import json
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR
from typing import Dict
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
#from onedal.svm import SVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder
import joblib

warnings.filterwarnings('ignore')

date_time = datetime.now().strftime("%d.%m.%Y, %H:%M:%S")

# Read CSV file to load data
df_wine = pd.read_csv('wine_validation.csv', sep=';', index_col='Unnamed: 0')

# Changing column names -> space to underscore
df_wine.columns = df_wine.columns.str.replace(' ', '_')

# Standard Preprocessing

- Missing Value Handling

In [6]:
df_wine = missing_value_handling(df_wine)

In [13]:
df_wine_clean = df_wine[df_wine.quality != 99]
df_wine_clean

,wine_type,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,magnesium,flavanoids,minerals,calcium,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,Pinot noir,5.8,0.15,0.49,1.1,76.729301,894.94,186.639301,109.91,0.048,21.0,98.0,0.99290,3.19,0.48,9.2,5
1,Merlot,6.6,0.25,0.32,5.6,4.795712,1160.95,251.875712,247.08,0.039,15.0,68.0,0.99163,2.96,0.52,11.1,6
2,Chardonnay,6.7,0.21,0.34,1.5,85.193710,789.82,304.703710,219.51,0.035,45.0,123.0,0.98949,3.24,0.36,12.6,7
3,Merlot,8.3,0.28,0.27,17.5,11.976525,777.86,237.586525,225.61,0.045,48.0,253.0,1.00014,3.02,0.56,9.1,6
4,Merlot,7.5,0.42,0.19,6.9,5.599673,785.72,95.399673,89.80,0.041,62.0,150.0,0.99508,3.23,0.37,10.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,Cabernet Sauvignon,6.4,0.23,0.37,7.9,92.701914,1143.32,318.791914,226.09,0.050,60.0,150.0,0.99488,2.86,0.49,9.3,6
3996,Cabernet Sauvignon,7.0,0.22,0.26,9.2,94.807955,863.32,322.107955,227.30,0.027,37.0,122.0,0.99228,3.06,0.34,12.5,8
3997,Merlot,7.5,0.26,0.30,4.6,50.112474,831.67,360.872474,310.76,0.027,29.0,92.0,0.99085,3.15,0.38,12.0,7
3998,Chardonnay,6.3,0.43,0.32,8.8,84.805688,1300.32,328.915688,244.11,0.042,18.0,106.0,0.99172,3.28,0.33,12.9,7


2. Separate Gamay and No-Gamay

In [14]:
df_wine_no_gamay = df_wine_clean[df_wine_clean["wine_type"]!="Gamay"].copy()
df_wine_gamay = df_wine_clean[df_wine_clean["wine_type"]=="Gamay"].copy()

3. OneHotEncode Wine Type for df_wine_clean, df_wine_no_gamay
4. Drop Wine Type for df_wine_gamay

In [15]:
from sklearn.preprocessing import OneHotEncoder

df_wine_gamay = df_wine_gamay.copy().drop(columns=["wine_type"])
df_wine_gamay

# One Hot Encoding
wine_type_array = df_wine_no_gamay["wine_type"].to_numpy()

ohe = OneHotEncoder(sparse=False)
wine_type_transformed = ohe.fit_transform(wine_type_array.reshape(-1, 1))  # 1d array -> 2d array
df_wine_no_gamay[ohe.categories_[0]] = wine_type_transformed
df_wine_no_gamay = df_wine_no_gamay.copy().drop(columns=["wine_type"])
#df_wine_no_gamay

# OneHotEncoding df_clean
wine_type_array = df_wine_clean["wine_type"].to_numpy()

ohe = OneHotEncoder(sparse=False)
wine_type_transformed = ohe.fit_transform(wine_type_array.reshape(-1, 1))  # 1d array -> 2d array
df_wine_clean[ohe.categories_[0]] = wine_type_transformed
df_wine_clean = df_wine_clean.copy().drop(columns=["wine_type"])
df_wine_clean

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,magnesium,flavanoids,minerals,calcium,chlorides,free_sulfur_dioxide,...,density,pH,sulphates,alcohol,quality,Cabernet Sauvignon,Chardonnay,Gamay,Merlot,Pinot noir
0,5.8,0.15,0.49,1.1,76.729301,894.94,186.639301,109.91,0.048,21.0,...,0.99290,3.19,0.48,9.2,5,0.0,0.0,0.0,0.0,1.0
1,6.6,0.25,0.32,5.6,4.795712,1160.95,251.875712,247.08,0.039,15.0,...,0.99163,2.96,0.52,11.1,6,0.0,0.0,0.0,1.0,0.0
2,6.7,0.21,0.34,1.5,85.193710,789.82,304.703710,219.51,0.035,45.0,...,0.98949,3.24,0.36,12.6,7,0.0,1.0,0.0,0.0,0.0
3,8.3,0.28,0.27,17.5,11.976525,777.86,237.586525,225.61,0.045,48.0,...,1.00014,3.02,0.56,9.1,6,0.0,0.0,0.0,1.0,0.0
4,7.5,0.42,0.19,6.9,5.599673,785.72,95.399673,89.80,0.041,62.0,...,0.99508,3.23,0.37,10.0,6,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,6.4,0.23,0.37,7.9,92.701914,1143.32,318.791914,226.09,0.050,60.0,...,0.99488,2.86,0.49,9.3,6,1.0,0.0,0.0,0.0,0.0
3996,7.0,0.22,0.26,9.2,94.807955,863.32,322.107955,227.30,0.027,37.0,...,0.99228,3.06,0.34,12.5,8,1.0,0.0,0.0,0.0,0.0
3997,7.5,0.26,0.30,4.6,50.112474,831.67,360.872474,310.76,0.027,29.0,...,0.99085,3.15,0.38,12.0,7,0.0,0.0,0.0,1.0,0.0
3998,6.3,0.43,0.32,8.8,84.805688,1300.32,328.915688,244.11,0.042,18.0,...,0.99172,3.28,0.33,12.9,7,0.0,1.0,0.0,0.0,0.0


3 standard dataframes to work with:

In [16]:
df_wine_clean
df_wine_no_gamay
df_wine_gamay

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,magnesium,flavanoids,minerals,calcium,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
9,7.70,0.28,0.39,8.90,54.450579,929.44,377.690579,323.24,0.036,8.0,117.0,0.99350,3.06,0.38,12.0,2
41,6.15,0.21,0.37,3.20,29.384680,1116.19,293.484680,264.10,0.021,20.0,80.0,0.99076,3.39,0.47,12.0,1
88,7.10,0.22,0.33,2.80,1.593610,1149.79,124.863610,123.27,0.033,48.0,153.0,0.98990,3.15,0.38,12.7,2
91,7.60,0.19,0.32,18.75,48.741901,969.51,375.291901,326.55,0.047,32.0,193.0,1.00014,3.10,0.50,9.3,2
108,6.60,0.22,0.37,15.40,52.194621,932.46,225.104621,172.91,0.035,62.0,153.0,0.99845,3.02,0.40,9.3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3757,7.00,0.12,0.29,10.30,47.214956,1482.12,182.594956,135.38,0.039,41.0,98.0,0.99564,3.19,0.38,9.8,2
3861,7.80,0.26,0.49,3.20,25.888285,1301.54,347.008285,321.12,0.027,28.0,87.0,0.99190,3.03,0.32,11.3,2
3893,7.90,0.18,0.37,3.00,45.470051,988.04,244.680051,199.21,0.061,25.0,178.0,0.99500,3.22,0.51,10.0,2
3911,7.00,0.24,0.25,1.70,28.317335,1295.30,217.397335,189.08,0.042,48.0,189.0,0.99200,3.25,0.42,11.4,2


### Use model

In [ ]:
# Test split
X = df_wine_clean.drop('quality', axis='columns')
y = df_wine_clean['quality']
# split off some data for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1.0)

In [ ]:
# loaded_model = joblib.load('finalized_model_gamay.sav')
loaded_model = joblib.load('finalized_model_no_gamay.sav')
result = loaded_model.score(X_test, y_test)
print(result)